In [ ]:
# 구글드라이브와 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
# label 데이터셋 확인
pd.read_csv("/content/drive/My Drive/project_dataset/dacon_m12/dirty_mnist_answer.csv")

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,1,1,0,1,0,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,1,1,1
1,1,0,1,0,1,1,0,0,1,1,1,0,0,1,1,0,0,1,0,1,1,0,1,1,0,0,1
2,2,1,0,1,0,1,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,1
3,3,0,1,1,1,0,0,0,1,1,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0
4,4,1,1,1,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,0,1,1,0,1,0,1,1,1,0,1,1,0,0,0,1,0,1,1,1,0,0,0,0,0,1
49996,49996,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1,1,1
49997,49997,1,0,0,1,0,0,1,1,1,1,0,0,0,0,0,1,0,1,0,1,1,1,0,0,0,1
49998,49998,1,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,1,1,1,1,1,0,1,1,0,0


In [ ]:
# 구동 디렉토리 경로변경
import os
os.chdir('/content/drive/MyDrive/project_dataset/dacon_m12')
!pwd

/content/drive/MyDrive/project_dataset/dacon_m12


In [ ]:
# # 현재 경로에 새로운 디렉토리 생성
# # 생성한 디렉토리에 zip파일 압축해제
# # 구글 드라이브가 맛탱이가 가서 재접속 할때마다 기존 폴더 삭제하고 다시 압축 풀어야함?

from google.colab import output

# 폴더 삭제하는 코드
import shutil
shutil.rmtree("dirty_mnist4", ignore_errors=True)
!mkdir "dirty_mnist4"
!unzip "dirty_mnist.zip" -d "dirty_mnist4"

# !mkdir "./test_dirty_mnist"
# !unzip "test_dirty_mnist.zip" -d "./test_dirty_mnist"

# !mkdir "./mnist_origin"
# !unzip "mnist_data.zip" -d "./mnist_origin"

output.clear()


In [ ]:
import tensorflow as tf
from tensorflow import  keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from PIL import Image

In [ ]:
# 데이터가 존재하는 경로

train_dir = "/content/drive/MyDrive/project_dataset/dacon_m12/dirty_mnist3"
# len(os.listdir("/content/drive/MyDrive/project_dataset/dacon_m12/dirty_mnist3"))

In [ ]:
meta_df

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,00000.png,1,1,0,1,0,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,1,1,1
1,00001.png,0,1,0,1,1,0,0,1,1,1,0,0,1,1,0,0,1,0,1,1,0,1,1,0,0,1
2,00002.png,1,0,1,0,1,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,1
3,00003.png,0,1,1,1,0,0,0,1,1,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0
4,00004.png,1,1,1,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995.png,0,1,1,0,1,0,1,1,1,0,1,1,0,0,0,1,0,1,1,1,0,0,0,0,0,1
49996,49996.png,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1,1,1
49997,49997.png,1,0,0,1,0,0,1,1,1,1,0,0,0,0,0,1,0,1,0,1,1,1,0,0,0,1
49998,49998.png,1,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,1,1,1,1,1,0,1,1,0,0


In [ ]:
meta_df = pd.read_csv('dirty_mnist_answer.csv')
meta_df['index'] = meta_df['index'].apply(lambda x: str("{0:05d}".format(x))+'.png')
# meta_df['index'] = meta_df['index'].astype("str").str.zfill(5) + '.png'
columns = list(meta_df.columns[1:])


datagen = ImageDataGenerator(rescale=1./255., validation_split=0.2)   # /255로 scaling - Standardization으로도 해볼것
train_gen_1 = datagen.flow_from_dataframe(dataframe = meta_df,        # 메인 데이터프레임
                                        directory = train_dir,        # 데이터 존재 경로
                                        x_col='index',                # 이미지 파일이름 column 명
                                        y_col=columns,                # 레이블들
                                        batch_size=64,                # batch 크기 64 
                                        seed=1,
                                        shuffle=True,
                                        color_mode = "rgb",           # RGB 사전훈련모델을 사용하기위해 rgb로 불러오기
                                        class_mode='raw',
                                        target_size=(256, 256),       # 이미지 shape 원본 그대로
                                        subset='training')
val_gen_1 = datagen.flow_from_dataframe(dataframe = meta_df,
                                        directory = train_dir,
                                        x_col='index',
                                        y_col=columns,
                                        batch_size=64,
                                        seed=1,
                                        shuffle=True,
                                        color_mode = "rgb",
                                        class_mode='raw',
                                        target_size=(256, 256),
                                        subset='validation')

Found 40000 validated image filenames.
Found 10000 validated image filenames.


In [ ]:
train_gen_2 = datagen.flow_from_dataframe(dataframe = meta_df,       
                                        directory = train_dir,       
                                        x_col='index',               
                                        y_col=columns,               
                                        batch_size=128,                     # 128
                                        seed=1,
                                        shuffle=True,
                                        color_mode = "grayscale",           # grayscale
                                        class_mode='raw',
                                        target_size=(128, 128),             # 128, 128
                                        subset='training')
val_gen_2 = datagen.flow_from_dataframe(dataframe = meta_df,
                                        directory = train_dir,
                                        x_col='index',
                                        y_col=columns,
                                        batch_size=128,
                                        seed=1,
                                        shuffle=True,
                                        color_mode = "grayscale",
                                        class_mode='raw',
                                        target_size=(128, 128),
                                        subset='validation')

Found 40000 validated image filenames.
Found 10000 validated image filenames.


In [ ]:
## xception

xcept = keras.Sequential([
                          keras.applications.xception.Xception(weights = "imagenet", 
                                                               include_top = False, 
                                                               input_shape = (256,256,3)),
                          keras.layers.GlobalAveragePooling2D(),
                          keras.layers.Dense(256, activation = 'relu'),
                          keras.layers.Dropout(0.3),
                          keras.layers.Dense(128, activation = 'relu'),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(26, kernel_initializer='he_normal', activation = "sigmoid", name='predictions')
])


optimizer = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
xcept.compile(optimizer = optimizer, loss="binary_crossentropy", metrics=["accuracy"])


checkpoint = keras.callbacks.ModelCheckpoint(f'./dacon_m12/m1_xception_0211.h5', monitor='val_accuracy', 
                                             save_best_only=True, verbose=1)
early_stop_cb = keras.callbacks.EarlyStopping(patience = 5, restore_best_weights = True)


xcept.fit_generator(generator = train_gen_1,
                    epochs = 10, steps_per_epoch = 40000//128,
                    validation_data = val_gen_1,
                    callbacks = [checkpoint, early_stop_cb])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
 76/312 [======>.......................] - ETA: 1:27:23 - loss: 0.6924 - accuracy: 0.0203

KeyboardInterrupt: ignored

In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

class SmallerVGGNet:
    @staticmethod
	  def build(width, height, depth, classes, finalAct="sigmoid"):
		    
		    model = Sequential()
		    inputShape = (256, 256, 3)
 
        # CONV => RELU => POOL
		    model.add(Conv2D(32, (5, ), padding="same",input_shape=inputShape))
		    model.add(Activation("relu"))
		    model.add(BatchNormalization())
		    model.add(MaxPooling2D(pool_size=(3, 3)))
		    model.add(Dropout(0.25))
        
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
    
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # FC => RELU
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
    
        model.add(Dense(classes))
        model.add(Activation(finalAct))
    
        return model

In [ ]:
model = SmallerVGGNet.build(
	width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
	depth=IMAGE_DIMS[2], classes=len(mlb.classes_),
	finalAct="sigmoid")

# 옵티마이저를 초기화합니다
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)


model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
 
# 네트워크를 학습시킵니다
print("[INFO] training network...")
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

In [ ]:
## resnet50v2

resnet50v2 = keras.Sequential([
                          keras.applications.ResNet50V2(weights = "imagenet", 
                                                               include_top = False, 
                                                               input_shape = (256,256,3)),
                          keras.layers.GlobalAveragePooling2D(),
                          keras.layers.Dense(256, activation = 'relu'),
                          keras.layers.Dropout(0.3),
                          keras.layers.Dense(128, activation = 'relu'),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(26, activation = "sigmoid", name='predictions')
])


optimizer = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
resnet50v2.compile(optimizer = optimizer, loss="binary_crossentropy", metrics=["accuracy"])


checkpoint = keras.callbacks.ModelCheckpoint(f'./dacon_m12/m1_resnet50v2_0211.h5', monitor='val_accuracy', 
                                             save_best_only=True, verbose=1)
early_stop_cb = keras.callbacks.EarlyStopping(patience = 5, restore_best_weights = True)


resnet50v2.fit_generator(generator = train_gen_1,
                    epochs = 10, 
                    validation_data = val_gen_1,
                    callbacks = [checkpoint, early_stop_cb])

In [ ]:
# simple

from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.layers import Activation, Convolution2D, BatchNormalization, Flatten, Dense, Dropout,MaxPool2D

model = Sequential()

# ( Conv => BN => LReLU ) * 2 => pool / 2 
model.add(Convolution2D(32, (7,7), padding='same', use_bias=False, input_shape=(128,128,1)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPool2D(pool_size=(2, 2)))

# ( Conv => BN => LReLU ) * 2 => pool / 2 
model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPool2D(pool_size=(2, 2)))

# ( Conv => BN => LReLU ) * 2 => pool / 2 
model.add(Convolution2D(128, (3,3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(Convolution2D(128, (3,3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(26, activation = "sigmoid"))


# complie
opt = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
model.compile(optimizer = opt, loss="binary_crossentropy", metrics=["accuracy"])

# callbacks
checkpoint = keras.callbacks.ModelCheckpoint(f'./dacon_m12/m1_simple.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop_cb = keras.callbacks.EarlyStopping(patience = 5, restore_best_weights = True)

# fit
model.fit_generator(generator = train_gen_2,
                    epochs = 10,
                    validation_data = val_gen_2,
                    callbacks = [checkpoint, early_stop_cb])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
 77/313 [======>.......................] - ETA: 2:24:09 - loss: 1.5329 - accuracy: 0.0212

KeyboardInterrupt: ignored

In [ ]:
meta_df = pd.read_csv('dirty_mnist_answer.csv')
meta_df['index'] = meta_df['index'].apply(lambda x: str("{0:05d}".format(x))+'.png')
# meta_df['index'] = meta_df['index'].astype("str").str.zfill(5) + '.png'
columns = list(meta_df.columns[1:])


datagen = ImageDataGenerator(rescale=1./255., validation_split=0.1)   # /255로 scaling - Standardization으로도 해볼것
train_gen = datagen.flow_from_dataframe(dataframe = meta_df,          # 메인 데이터프레임
                                        directory = train_dir,        # 데이터 존재 경로
                                        x_col='index',                # 이미지 파일이름이 들어간 column 이름
                                        y_col=columns,                # 레이블들
                                        batch_size = 128,             # 배치 사이즈 128
                                        seed=1,
                                        shuffle=True,
                                        color_mode = "rgb",           # RGB 사전훈련모델을 사용하기위해 rgb로 불러오기
                                        class_mode='raw',
                                        target_size=(128, 128),       # 해상도 down
                                        subset='training')
val_gen = datagen.flow_from_dataframe(dataframe = meta_df,
                                        directory = train_dir,
                                        x_col='index',
                                        y_col=columns,
                                        batch_size = 128,
                                        seed=1,
                                        shuffle=True,
                                        color_mode = "rgb",
                                        class_mode='raw',
                                        target_size=(256, 256),
                                        subset='validation')